In [49]:
import numpy as np
import pandas as pd
import itertools 
import matplotlib.pyplot as plt
import math

In [50]:
df_bancolombia = pd.read_csv("bancolombia.csv")
df_bancolombia

,Date,Price,Open,High,Low,Vol.,Change %
0,03/01/2024,"33,000.0","32,860.0","33,400.0","32,860.0",176.65K,0.18%
1,02/29/2024,"32,940.0","33,420.0","34,000.0","32,940.0",781.41K,-1.61%
2,02/28/2024,"33,480.0","33,480.0","34,540.0","33,480.0",100.06K,0.00%
3,02/27/2024,"33,480.0","33,440.0","33,740.0","33,380.0",140.23K,-0.06%
4,02/26/2024,"33,500.0","34,000.0","34,320.0","33,500.0",200.64K,-1.35%
...,...,...,...,...,...,...,...
949,04/07/2020,"26,000.0","26,000.0","27,020.0","25,760.0",421.29K,2.36%
950,04/06/2020,"25,400.0","24,400.0","25,900.0","24,020.0",305.28K,6.72%
951,04/03/2020,"23,800.0","23,940.0","23,940.0","23,460.0",446.59K,-4.42%
952,04/02/2020,"24,900.0","23,500.0","24,900.0","23,080.0",192.25K,3.75%


# Preprocesamiento

In [51]:
#Eliminar columnas inútiles
df_bancolombia = df_bancolombia.drop(columns=['Open','High','Low','Vol.','Change %'])

#Se invierte el dataframe para empezar desde los datos mas antiguos
df_bancolombia = df_bancolombia.iloc[::-1]
df_bancolombia = df_bancolombia.reset_index(drop=True)

#Convertir los precios de string a float
df_bancolombia['Price'] = df_bancolombia['Price'].str.replace(',', '')
df_bancolombia['Price'] = df_bancolombia['Price'].astype(float)

#Añade la columna Y_n definida en la guía
new_col = [0]
for i in range(1, len(df_bancolombia)):
    new_col.append(float((df_bancolombia['Price'][i]/df_bancolombia['Price'][i-1]) - 1))
df_bancolombia.insert(2, 'Yn', new_col, True)

#Eliminamos primer dato debido a que no se puede calcular Yn
df_bancolombia = df_bancolombia.drop([0])
df_bancolombia = df_bancolombia.reset_index(drop=True)

In [52]:
# Temporalmente ajusta las opciones de visualización para el DataFrame df_bancolombia
with pd.option_context('display.max_rows', None,
                       'display.max_columns', None,
                       'display.precision', 3,
                       ):
    print(df_bancolombia)

           Date    Price         Yn
0    04/02/2020  24900.0  3.750e-02
1    04/03/2020  23800.0 -4.418e-02
2    04/06/2020  25400.0  6.723e-02
3    04/07/2020  26000.0  2.362e-02
4    04/08/2020  26100.0  3.846e-03
5    04/13/2020  26960.0  3.295e-02
6    04/14/2020  26720.0 -8.902e-03
7    04/15/2020  26800.0  2.994e-03
8    04/16/2020  26880.0  2.985e-03
9    04/17/2020  26500.0 -1.414e-02
10   04/20/2020  26200.0 -1.132e-02
11   04/21/2020  24800.0 -5.344e-02
12   04/22/2020  24780.0 -8.065e-04
13   04/23/2020  24500.0 -1.130e-02
14   04/24/2020  23040.0 -5.959e-02
15   04/27/2020  24500.0  6.337e-02
16   04/28/2020  25500.0  4.082e-02
17   04/29/2020  25020.0 -1.882e-02
18   04/30/2020  25320.0  1.199e-02
19   05/04/2020  24140.0 -4.660e-02
20   05/05/2020  23720.0 -1.740e-02
21   05/06/2020  23000.0 -3.035e-02
22   05/07/2020  24240.0  5.391e-02
23   05/08/2020  24000.0 -9.901e-03
24   05/11/2020  23600.0 -1.667e-02
25   05/12/2020  23500.0 -4.237e-03
26   05/13/2020  22260.0 -5.

# Definición de estados

In [53]:
#calcula estados basados en el valor de la columna 'Yn' en relación con la desviación estándar de una ventana de datos de tamaño l
new_col = []
l = 20
data = df_bancolombia['Yn']

for i, percentage in enumerate(data):
    if i == len(data) - 1:
        min = i - l + 1
        desviacionA = np.std(data[min:i])

    if percentage >= 0:
        new_col.append('sube')
    else:
        new_col.append('baja')

# Inserta la columna de estados calculados en la posición 2 del DataFrame df_bancolombia
df_bancolombia.insert(2, 'Estado', new_col, True)
# Reestablece los índices del DataFrame df_bancolombia después de eliminar las filas
df_bancolombia = df_bancolombia.reset_index(drop=True)

In [54]:
df_bancolombia

,Date,Price,Estado,Yn
0,04/02/2020,24900.0,sube,0.037500
1,04/03/2020,23800.0,baja,-0.044177
2,04/06/2020,25400.0,sube,0.067227
3,04/07/2020,26000.0,sube,0.023622
4,04/08/2020,26100.0,sube,0.003846
...,...,...,...,...
948,02/26/2024,33500.0,baja,-0.013545
949,02/27/2024,33480.0,baja,-0.000597
950,02/28/2024,33480.0,sube,0.000000
951,02/29/2024,32940.0,baja,-0.016129


In [55]:
estados_posibles = ['baja','sube']  # Posibles estados del sistema

total_cols = len(estados_posibles)  # Calcula el número total de columnas, que es igual al número de estados posibles
total_rows = total_cols * total_cols  # Calcula el número total de filas como el cuadrado del número de estados posibles


In [56]:
#matriz de transición para un modelo Markoviano
estados = np.array(df_bancolombia['Estado'])

markovian_table = np.array(estados)
estados = np.delete(estados, 0)
estados = np.append(estados, 0)
markovian_table = np.vstack([markovian_table,estados])
estados = np.delete(estados, 0)
estados = np.append(estados, 0)
markovian_table = np.vstack([markovian_table,estados])

markovian_table = markovian_table.T
markovian_table = np.delete(markovian_table, [-1, -2], axis=0)
markovian_table 

array([['sube', 'baja', 'sube'],
       ['baja', 'sube', 'sube'],
       ['sube', 'sube', 'sube'],
       ...,
       ['baja', 'baja', 'sube'],
       ['baja', 'sube', 'baja'],
       ['sube', 'baja', 'sube']], dtype=object)

In [57]:
#frecuencias esperadas de transición entre estados
expected_freq = [[0] * (total_cols + 1) for i in range(total_cols)]

for i in range(len(markovian_table)):
    actual = markovian_table[i][1]
    row_index = estados_posibles.index(actual)
    col_index = estados_posibles.index(markovian_table[i][2])

    expected_freq[row_index][col_index] += 1
    expected_freq[row_index][total_cols] += 1

cols = estados_posibles.copy()
cols.append('Recuento')
expected_freq_df = pd.DataFrame(expected_freq, columns=cols)
cols.pop(-1)
expected_freq_df.insert(0, 'T_Actual', cols)
expected_freq_df

,T_Actual,baja,sube,Recuento
0,baja,225,232,457
1,sube,231,263,494


# Modelo - Matriz de transición

In [58]:
# probabilidad de transición entre los estados del modelo Markoviano
model_freq = [[0] * total_cols for i in range(total_cols)]
total_freq = [0,0,0,0,0,0,0,0]

for i in range(len(markovian_table)):
    actual = markovian_table[i][1]
    row_index = estados_posibles.index(actual)
    col_index = estados_posibles.index(markovian_table[i][2])

    model_freq[row_index][col_index] += 1
    total_freq[row_index] += 1

model_transition_prob_df = pd.DataFrame(model_freq, columns=estados_posibles)
model_transition_prob_df.insert(0, 'Actual', estados_posibles)

for i in range(total_cols):
    for j in range(1,total_cols+1):
        model_transition_prob_df.iat[i, j] = (model_transition_prob_df.iat[i, j]/total_freq[i]).astype(float)

model_transition_prob_df

C:\Users\FAMILIA\AppData\Local\Temp\ipykernel_25028\868081643.py:18: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.4923413566739606' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  model_transition_prob_df.iat[i, j] = (model_transition_prob_df.iat[i, j]/total_freq[i]).astype(float)
C:\Users\FAMILIA\AppData\Local\Temp\ipykernel_25028\868081643.py:18: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.5076586433260394' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  model_transition_prob_df.iat[i, j] = (model_transition_prob_df.iat[i, j]/total_freq[i]).astype(float)


,Actual,baja,sube
0,baja,0.492341,0.507659
1,sube,0.467611,0.532389


In [59]:
print("La última desviación medida es de: " + str(desviacionA))

La última desviación medida es de: 0.029178195582031243


In [60]:
A = [[0.85,0.15],[0.1,0.9]]
B = [[0.3,0.7],[0.5,0.5]]
total = [0,0,0,0]

for i in range(2):
    for j in range(2):
        a=A[i][j]
        for x in range(2):
            for y in range(2):
                b=B[x][y]
                total[i + 2*x] += a*b

total

[1.0, 1.0, 0.9999999999999999, 1.0]